In [1]:
import numpy as np 
import pandas as pd 
import re
import os
import glob
import pickle
from sklearn.model_selection import train_test_split 
import librosa
import random

In [2]:
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import concatenate
from keras import optimizers
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [3]:
INPUT_FOLDER = "input/"
# INPUT_FOLDER = "../input/"
print(os.listdir(INPUT_FOLDER))

['train_curated.csv', 'train_noisy.csv', 'sample_submission.csv', 'train_noisy', 'train_curated', 'test']


In [4]:
TRAIN_CURATED_PATH = INPUT_FOLDER + "train_curated.csv"
TRAIN_NOISY_PATH = INPUT_FOLDER + "train_noisy.csv"
SAMPLE_SUBMISSION_PATH = INPUT_FOLDER + "sample_submission.csv"
TRAIN_CURATED = INPUT_FOLDER + "train_curated/"
TRAIN_NOISY = INPUT_FOLDER + "train_noisy/"
TEST = INPUT_FOLDER + "test/"

train_curated = pd.read_csv(TRAIN_CURATED_PATH)
train_noisy = pd.read_csv(TRAIN_NOISY_PATH)
sample = pd.read_csv(SAMPLE_SUBMISSION_PATH)

In [5]:
def one_hot(labels, src_dict):
    ar = np.zeros([len(labels), len(src_dict)])
    for i, label in enumerate(labels):
        label_list = label.split(',')
        for la in label_list:
            ar[i, src_dict[la]] = 1
    return ar

In [6]:
target_names = sample.columns[1:]
num_targets = len(target_names)

src_dict = {target_names[i]:i for i in range(num_targets)}
src_dict_inv = {i:target_names[i] for i in range(num_targets)}

In [7]:
num_freq = 128
len_div = 256

In [8]:
inputs = Input(shape=(num_freq,len_div,1), name='input')

dense_list = []

## Block 1
conv1 = Conv2D(4, (19, 19),activation='relu',padding='same',name='conv1')(inputs)
pool1 = MaxPooling2D((19, 19),strides=(1, 1),padding='same',name='pool1')(conv1)
norm1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1')(pool1)
drop1 = Dropout(rate=0.05)(norm1)

conv1_1 = Conv2D(4, (11, 11),activation='relu',padding='same',name='conv1_1')(drop1)
pool1_1 = MaxPooling2D((5, 5),strides=(5, 5),padding='same',name='pool1_1')(conv1_1)
norm1_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm1_1')(pool1_1)
drop1_1 = Dropout(rate=0.05)(norm1_1)

flatten1 = Flatten(name='flatten1')(drop1)
dense1 = Dense(16, name='dense1')(flatten1)
act1 = Activation('relu',name='act1')(dense1)
dense_list.append(act1)

## Block 2
conv2 = Conv2D(4, (13, 13),activation='relu',padding='same',name='conv2')(inputs)
pool2 = MaxPooling2D((13, 13), strides=(1, 1), padding='same',name='pool2')(conv2)
norm2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2')(pool2)
drop2 = Dropout(rate=0.05)(norm2)

conv2_1 = Conv2D(4, (7, 7),activation='relu',padding='same',name='conv2_1')(drop2)
pool2_1 = MaxPooling2D((7, 7), strides=(5, 5), padding='same',name='pool2_1')(conv2_1)
norm2_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001,name='norm2_1')(pool2_1)
drop2_1 = Dropout(rate=0.05)(norm2_1)

flatten2 = Flatten(name='flatten2')(drop2_1)
dense2 = Dense(16, name='dense2')(flatten2)
act2 = Activation('relu',name='act2')(dense2)
dense_list.append(act2)

## Block 3
conv3 = Conv2D(8, (11, 11), activation='relu',padding='same',name='conv3')(inputs)
pool3 = MaxPooling2D((11, 11), strides=(2, 2), padding='same',name='pool3')(conv3)
norm3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm3')(pool3)
drop3 = Dropout(rate=0.05)(norm3)

conv3_1 = Conv2D(8, (5, 5), activation='relu',padding='same',name='conv3_1')(drop3)
pool3_1 = MaxPooling2D((5, 5), strides=(2, 2), padding='same',name='pool3_1')(conv3_1)
norm3_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm3_1')(pool3_1)
drop3_1 = Dropout(rate=0.05)(norm3_1)

flatten3 = Flatten(name='flatten3')(drop3_1)
dense3 = Dense(16, name='dense3')(flatten3)
act3 = Activation('relu',name='act3')(dense3)
dense_list.append(act3)

## Block 4
conv4 = Conv2D(8, (9, 9),activation='relu',padding='same',name='conv4')(inputs)
pool4 = MaxPooling2D((9, 9), strides=(2, 2), padding='same',name='pool4')(conv4)
norm4 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm4')(pool4)
drop4 = Dropout(rate=0.05)(norm4)

conv4_1 = Conv2D(8, (3, 3),activation='relu',padding='same',name='conv4_1')(drop4)
pool4_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='same',name='pool4_1')(conv4_1)
norm4_1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.0001,name='norm4_1')(pool4_1)
drop4_1 = Dropout(rate=0.05)(norm4_1)

flatten4 = Flatten(name='flatten4')(drop4_1)
dense4 = Dense(16, name='dense4')(flatten4)
act4 = Activation('relu',name='act4')(dense4)
dense_list.append(act4)

concat = concatenate(dense_list, name='concat', axis=1)

dense2 = Dense(80, name='dense_all')(concat)
pred = Activation('softmax',name='pred')(dense2)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model = Model(inputs=inputs, outputs=pred)

In [9]:
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
datagen = ImageDataGenerator(
           rotation_range=0,
           width_shift_range=16,
           height_shift_range=0,
           shear_range=0,
           zoom_range=0,
           horizontal_flip=False,
           vertical_flip=False)

In [11]:
# file_name = train_curated['fname'].values
# y_proc_tmp = one_hot(train_curated['labels'], src_dict)

# X_proc_ = np.zeros([1, num_freq, len_div])
# y_proc_ = np.zeros([1,80])
# for i, file in enumerate(file_name):
#     wavfile = TRAIN_CURATED + file
#     y_proc, sr = librosa.load(wavfile)
#     S = librosa.feature.melspectrogram(y_proc, sr=sr, n_mels=num_freq)
#     log_S = librosa.power_to_db(S, ref=np.max)
#     X_proc = (log_S + 80) / 40 - 1

#     num_div = X_proc.shape[1] // len_div
#     num_pad = len_div - X_proc.shape[1] % len_div
#     redidual_amp = np.zeros([num_freq, num_pad])
#     dum = np.hstack([X_proc, redidual_amp])
#     X_proc_ = np.vstack([X_proc_, np.array(np.split(dum, num_div+1,1))])
#     for _ in range(num_div+1):
#         y_proc_ = np.vstack([y_proc_, y_proc_tmp[i+div*k]])

# X_train = X_proc_[1:]
# y_train = y_proc_[1:]
# X_train = X_train.reshape([-1, num_freq, len_div, 1])

# print(X_train.shape)
# print(y_train.shape)

In [12]:
with open('train_arr.pickle', 'rb') as f:
    X_train = pickle.load(f)
    y_train = pickle.load(f)

In [13]:
datagen.fit(X_train)
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=10)

Epoch 1/10
294/293 [==============================] - 114s 389ms/step - loss: 6.3323 - acc: 0.0451
Epoch 2/10
294/293 [==============================] - 111s 378ms/step - loss: 5.1486 - acc: 0.0892
Epoch 3/10
294/293 [==============================] - 111s 379ms/step - loss: 4.5027 - acc: 0.1244
Epoch 4/10
294/293 [==============================] - 112s 381ms/step - loss: 4.2593 - acc: 0.1481
Epoch 5/10
294/293 [==============================] - 112s 382ms/step - loss: 3.9094 - acc: 0.1819
Epoch 6/10
294/293 [==============================] - 112s 381ms/step - loss: 3.7550 - acc: 0.1935
Epoch 7/10
294/293 [==============================] - 112s 380ms/step - loss: 3.5741 - acc: 0.2263
Epoch 8/10
294/293 [==============================] - 112s 381ms/step - loss: 3.4044 - acc: 0.2536
Epoch 9/10
294/293 [==============================] - 111s 378ms/step - loss: 3.2498 - acc: 0.2807
Epoch 10/10
294/293 [==============================] - 110s 376ms/step - loss: 3.1492 - acc: 0.2981


In [27]:
model1 = model
datagen.fit(X_train)
model1.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=10)

Epoch 1/10
46/45 [==============================] - 19s 407ms/step - loss: 3.3391 - acc: 0.3351
Epoch 2/10
12/45 [======>.......................] - ETA: 12s - loss: 3.2306 - acc: 0.3151

KeyboardInterrupt: 

In [14]:
file_name = train_noisy['fname'].values

div = 500
num_batch = len(file_name) // div
rest = len(file_name) % div

pos = [range(div*k, div*(k+1)) for k in range(num_batch)]
pos.append(range(div*num_batch, div*num_batch+rest))

y_proc_tmp = one_hot(train_noisy['labels'], src_dict)

epochs = 5

for e in range(epochs):
    k_list = random.sample(range(num_batch+1), num_batch+1)
    for k in k_list:
    #     X_proc_ = np.zeros([1, num_freq, len_div])
    #     y_proc_ = np.zeros([1,80])
    #     for i, file in enumerate(file_name[pos[k]]):
    #         wavfile = TRAIN_NOISY + file
    #         y_proc, sr = librosa.load(wavfile)
    #         S = librosa.feature.melspectrogram(y_proc, sr=sr, n_mels=num_freq)
    #         log_S = librosa.power_to_db(S, ref=np.max)
    #         X_proc = (log_S + 80) / 40 - 1

    #         num_div = X_proc.shape[1] // len_div
    #         num_pad = len_div - X_proc.shape[1] % len_div
    #         redidual_amp = np.zeros([num_freq, num_pad])
    #         dum = np.hstack([X_proc, redidual_amp])
    #         X_proc_ = np.vstack([X_proc_, np.array(np.split(dum, num_div+1,1))])
    #         for _ in range(num_div+1):
    #             y_proc_ = np.vstack([y_proc_, y_proc_tmp[i+div*k]])

    #     X_train = X_proc_[1:]
    #     y_train = y_proc_[1:]
    #     X_train = X.reshape([-1, num_freq, len_div, 1])

        with open('out/train_noisy_arr_{}.pickle'.format(k), 'rb') as f:
            X_train = pickle.load(f)
            y_train = pickle.load(f)

        datagen.fit(X_train)
        model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                            steps_per_epoch=len(X_train) / 32, epochs=3)
print('noisy iter epoch No.{} is done.'.format(e))

Epoch 1/3
46/45 [==============================] - 18s 393ms/step - loss: 5.3930 - acc: 0.0557
Epoch 2/3
46/45 [==============================] - 17s 377ms/step - loss: 4.6812 - acc: 0.0773
Epoch 3/3
46/45 [==============================] - 17s 377ms/step - loss: 4.4690 - acc: 0.1046
Epoch 1/3
46/45 [==============================] - 18s 398ms/step - loss: 4.8179 - acc: 0.0920
Epoch 2/3
46/45 [==============================] - 17s 379ms/step - loss: 4.5058 - acc: 0.1103
Epoch 3/3
46/45 [==============================] - 17s 379ms/step - loss: 4.2733 - acc: 0.1381
Epoch 1/3
46/46 [==============================] - 18s 382ms/step - loss: 4.7279 - acc: 0.1033
Epoch 2/3
46/46 [==============================] - 18s 382ms/step - loss: 4.4133 - acc: 0.1250
Epoch 3/3
46/46 [==============================] - 18s 381ms/step - loss: 4.1513 - acc: 0.1501
Epoch 1/3
47/46 [==============================] - 18s 378ms/step - loss: 4.7435 - acc: 0.0858
Epoch 2/3
47/46 [==============================] -

46/45 [==============================] - 18s 382ms/step - loss: 3.4849 - acc: 0.2489
Epoch 1/3
46/45 [==============================] - 18s 381ms/step - loss: 4.0970 - acc: 0.1819
Epoch 2/3
46/45 [==============================] - 18s 381ms/step - loss: 3.8070 - acc: 0.2089
Epoch 3/3
46/45 [==============================] - 18s 382ms/step - loss: 3.6417 - acc: 0.2325
Epoch 1/3
46/45 [==============================] - 18s 381ms/step - loss: 4.0251 - acc: 0.1872
Epoch 2/3
46/45 [==============================] - 18s 381ms/step - loss: 3.7012 - acc: 0.2336
Epoch 3/3
46/45 [==============================] - 17s 380ms/step - loss: 3.5143 - acc: 0.2612
Epoch 1/3
46/45 [==============================] - 17s 380ms/step - loss: 4.0962 - acc: 0.1826
Epoch 2/3
46/45 [==============================] - 17s 379ms/step - loss: 3.7448 - acc: 0.2258
Epoch 3/3
46/45 [==============================] - 17s 379ms/step - loss: 3.5721 - acc: 0.2527
Epoch 1/3
46/45 [==============================] - 18s 382ms

47/46 [==============================] - 18s 375ms/step - loss: 3.8859 - acc: 0.2155
Epoch 3/3
47/46 [==============================] - 18s 375ms/step - loss: 3.6938 - acc: 0.2567
Epoch 1/3
46/45 [==============================] - 17s 380ms/step - loss: 4.0153 - acc: 0.2017
Epoch 2/3
46/45 [==============================] - 17s 380ms/step - loss: 3.7050 - acc: 0.2493
Epoch 3/3
46/45 [==============================] - 18s 381ms/step - loss: 3.5605 - acc: 0.2721
Epoch 1/3
46/45 [==============================] - 18s 382ms/step - loss: 3.9814 - acc: 0.2171
Epoch 2/3
46/45 [==============================] - 18s 381ms/step - loss: 3.6819 - acc: 0.2368
Epoch 3/3
46/45 [==============================] - 18s 381ms/step - loss: 3.5078 - acc: 0.2679
Epoch 1/3
46/45 [==============================] - 17s 380ms/step - loss: 3.8821 - acc: 0.2146
Epoch 2/3
46/45 [==============================] - 17s 379ms/step - loss: 3.6077 - acc: 0.2614
Epoch 3/3
46/45 [==============================] - 17s 379ms

46/45 [==============================] - 17s 379ms/step - loss: 3.8574 - acc: 0.2011
Epoch 2/3
46/45 [==============================] - 17s 380ms/step - loss: 3.5716 - acc: 0.2516
Epoch 3/3
46/45 [==============================] - 17s 379ms/step - loss: 3.4119 - acc: 0.2665
Epoch 1/3
47/46 [==============================] - 18s 378ms/step - loss: 4.1224 - acc: 0.1906
Epoch 2/3
47/46 [==============================] - 18s 377ms/step - loss: 3.8187 - acc: 0.2268
Epoch 3/3
47/46 [==============================] - 18s 376ms/step - loss: 3.6999 - acc: 0.2388
Epoch 1/3
46/45 [==============================] - 18s 382ms/step - loss: 3.6711 - acc: 0.2259
Epoch 2/3
46/45 [==============================] - 18s 382ms/step - loss: 3.4050 - acc: 0.2635
Epoch 3/3
46/45 [==============================] - 18s 382ms/step - loss: 3.2645 - acc: 0.2785
Epoch 1/3
46/45 [==============================] - 17s 380ms/step - loss: 4.0315 - acc: 0.1868
Epoch 2/3
46/45 [==============================] - 17s 380ms

46/45 [==============================] - 17s 380ms/step - loss: 3.5247 - acc: 0.2746
Epoch 1/3
46/45 [==============================] - 18s 382ms/step - loss: 3.8347 - acc: 0.2300
Epoch 2/3
46/45 [==============================] - 17s 380ms/step - loss: 3.5677 - acc: 0.2540
Epoch 3/3
46/45 [==============================] - 17s 379ms/step - loss: 3.4575 - acc: 0.2771
Epoch 1/3
46/45 [==============================] - 18s 381ms/step - loss: 3.9598 - acc: 0.2131
Epoch 2/3
46/45 [==============================] - 17s 380ms/step - loss: 3.6555 - acc: 0.2683
Epoch 3/3
46/45 [==============================] - 18s 381ms/step - loss: 3.5133 - acc: 0.2953
Epoch 1/3
47/46 [==============================] - 18s 375ms/step - loss: 3.8404 - acc: 0.2002
Epoch 2/3
47/46 [==============================] - 18s 374ms/step - loss: 3.7134 - acc: 0.2282
Epoch 3/3
47/46 [==============================] - 18s 374ms/step - loss: 3.4877 - acc: 0.2581
Epoch 1/3
46/45 [==============================] - 18s 382ms

46/45 [==============================] - 18s 386ms/step - loss: 3.7468 - acc: 0.2305
Epoch 3/3
46/45 [==============================] - 18s 387ms/step - loss: 3.6072 - acc: 0.2502
Epoch 1/3
47/46 [==============================] - 18s 384ms/step - loss: 3.9709 - acc: 0.1909
Epoch 2/3
47/46 [==============================] - 18s 383ms/step - loss: 3.6674 - acc: 0.2391
Epoch 3/3
47/46 [==============================] - 18s 383ms/step - loss: 3.5538 - acc: 0.2694
Epoch 1/3
46/45 [==============================] - 18s 389ms/step - loss: 3.8247 - acc: 0.2187
Epoch 2/3
46/45 [==============================] - 18s 389ms/step - loss: 3.5727 - acc: 0.2621
Epoch 3/3
46/45 [==============================] - 18s 392ms/step - loss: 3.4082 - acc: 0.3011
Epoch 1/3
46/45 [==============================] - 18s 391ms/step - loss: 3.7942 - acc: 0.2204
Epoch 2/3
46/45 [==============================] - 18s 390ms/step - loss: 3.5215 - acc: 0.2696
Epoch 3/3
46/45 [==============================] - 18s 390ms

46/45 [==============================] - 18s 388ms/step - loss: 3.9668 - acc: 0.1890
Epoch 2/3
46/45 [==============================] - 18s 387ms/step - loss: 3.6574 - acc: 0.2531
Epoch 3/3
46/45 [==============================] - 18s 387ms/step - loss: 3.5173 - acc: 0.2590
Epoch 1/3
46/45 [==============================] - 18s 387ms/step - loss: 3.8348 - acc: 0.2299
Epoch 2/3
46/45 [==============================] - 18s 386ms/step - loss: 3.5856 - acc: 0.2887
Epoch 3/3
46/45 [==============================] - 18s 385ms/step - loss: 3.4234 - acc: 0.3068
Epoch 1/3
46/45 [==============================] - 18s 387ms/step - loss: 3.7049 - acc: 0.2396
Epoch 2/3
46/45 [==============================] - 18s 386ms/step - loss: 3.4506 - acc: 0.2710
Epoch 3/3
46/45 [==============================] - 18s 387ms/step - loss: 3.2720 - acc: 0.3105
Epoch 1/3
47/46 [==============================] - 18s 380ms/step - loss: 3.6050 - acc: 0.2441
Epoch 2/3
47/46 [==============================] - 18s 378ms

In [15]:
with open('20190506model_curated_noisy.pickle', 'wb') as f:
    pickle.dump(model, f)

In [16]:
with open('20190506model_curated_noisy.pickle', 'rb') as f:
    model = pickle.load(f)

In [17]:
X_test_list = []

filename = glob.glob(TEST + "*")

for file in filename:
    wavfile = file
    y_proc, sr = librosa.load(wavfile)
    S = librosa.feature.melspectrogram(y_proc, sr=sr, n_mels=num_freq)
    log_S = librosa.power_to_db(S, ref=np.max)
    X_proc = (log_S + 80) / 40 - 1
    
    num_div = X_proc.shape[1] // len_div
    num_pad = len_div - X_proc.shape[1] % len_div
    redidual_amp = np.zeros([num_freq, num_pad])
    dum = np.hstack([X_proc, redidual_amp])
    X_test_list.append(np.array(np.split(dum, num_div+1,1)))
    

In [18]:
with open('out/test_arr.pickle', 'wb') as f:
    pickle.dump(X_test_list, f)
    pickle.dump(filename, f)

In [19]:
with open('out/test_arr.pickle', 'rb') as f:
    X_test_list = pickle.load(f)
    filename = pickle.load(f)

In [20]:
# X_test_list = []

# for file in filename:
#     wavfile = file
#     y_proc, sr = librosa.load(wavfile)
#     S = librosa.feature.melspectrogram(y_proc, sr=sr, n_mels=num_freq)
#     log_S = librosa.power_to_db(S, ref=np.max)
#     X_proc = (log_S + 80) / 40 - 1
    
#     num_div = X_proc.shape[1] // len_div
#     num_pad = len_div - X_proc.shape[1] % len_div
#     redidual_amp = np.zeros([num_freq, num_pad])
#     dum = np.hstack([X_proc, redidual_amp])
#     X_test_list.append(np.array(np.split(dum, num_div+1,1)))

In [21]:
pred_list = []
for X_test in X_test_list:
    pred = model.predict(X_test.reshape([-1, num_freq, len_div,1])).sum(axis=0) / len(X_test)
    pred_list.append(pred)
y_pred = np.array(pred_list)

In [22]:
sound_names = sample.columns[1:]

In [23]:
names = []
for f in filename:
    names.append(f.split('/')[-1])

In [24]:
se_file = pd.Series(names, name='fname')
label = pd.DataFrame(y_pred, columns=sound_names)

In [25]:
sub_df = pd.concat([se_file, label], axis=1)

In [26]:
sub_df.to_csv('submission.csv', index=False)